In [1]:
import random
# Function to generate random unfriendly relationship matrix
def generate_unfriendly_matrix(num_people, num_unfriendly, groups): # TODO
    matrix = [[0 for _ in range(num_people)] for _ in range(num_people)]
    while num_unfriendly > 0:
        i = random.randint(0, num_people - 1)
        j = random.randint(0, num_people - 1)
        # Ensure i != j and avoid duplicate unfriendly relationships
        if i != j and matrix[i][j] == 0:
            flag = False
            for group in groups:
                if i in group and j in group:
                    flag = True
                    break
            if not flag:
                matrix[i][j] = 1
                matrix[j][i] = 1 # Make it symmetric
                num_unfriendly -= 1
    # print("training data finished")
    return matrix

def create_groups(num_people, group_size):
    groups = []
    num_groups = num_people // group_size
    # Loop through the number of groups
    for _ in range(num_groups):
        # Create a new group list
        group = []

        # Fill the group with people (up to group_size)
        for _ in range(group_size):
            if num_people > 0:  # from 1 to num_people
                group.append(num_people)  # Add person names (example)
                num_people -= 1

        # Add the filled group to the groups list
        groups.append(group)

    return groups

def calculate_conflict_cost(seating_arrangement, unfriendly_matrix, groups):
    # print(seating_arrangement)
    alpha = 0.5
    conflict_cost = 0
    # conflicts
    for i in range(len(seating_arrangement)):
        for j in range(i + 1, len(seating_arrangement)): # Avoid double counting
            distance = abs(i - j)
            weight = 1.0 / (distance**2 + 1) # Avoid division by zero and give higher weight to closer positions
            cost = unfriendly_matrix[seating_arrangement[i]][seating_arrangement[j]] * weight
            conflict_cost += cost
    # groups

    neighbering_cost = 0
    for i in range(len(groups)):
        group = groups[i]
        # neighbering_cost = 0
        dictance = 0
        for j in range(len(group)):
            for k in range(j+1, len(group)):
                j_inx = seating_arrangement.index(j)
                k_inx = seating_arrangement.index(k)
                distance += abs(k_inx - j_inx - 1)
        if distance > 10: # sum (index difirences)
            neighbering_cost += - 1.0 / ((distance - 10 + 1) ** 2 + 1)

    total_cost = alpha * conflict_cost + (1 - alpha) * neighbering_cost
    return total_cost


In [2]:

# generate training data
def generate_training_data(num_people, num_unfriendly, num_samples, groups):
    training_data = []
    for _ in range(num_samples):
        _unfriendly_matrix = generate_unfriendly_matrix(num_people, num_unfriendly, groups)
        training_data.append(_unfriendly_matrix)
    return training_data

def convert_to_permutation(output):
    _, indices = torch.sort(output, descending=True)
    return indices + 1  # تبدیل به دامنه 1 تا 24

def convert_index_to_names(predicted_seating_arrangement, num_people, row, col):
    first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya", "Razie", "Masoome", "Kosar", "Bahare", "Amir", "AmirHosein", "AmirMahdi", "Hassan", "Abbas", "Abolfazl", "AmirAbbas"
    "Hashem", "Vesal", "Zeynab", "Saeide", "Sobhan", "Khadije", "Hamid", "Reza"]
    last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam", "Ebrahimi", "Arjmandi", "Erfani", "Jalali", "boushehri",
                  "Shirazi", "Hassani", "Rokni"]
    # random.seed(40)
    random_names = set()
    while len(random_names) < num_people:
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        random_names.add(f"{first_name} {last_name}")

    random_names = list(random_names) # primary list of names
    seating_arrangement = [[0 for _ in range(col)] for _ in range(row)]
    t = 0

    for i in range(row):
        for j in range(col):
            seating_arrangement[i][j] = random_names[predicted_seating_arrangement[t]]
            t += 1
    return seating_arrangement


def _print(seating_arrangement):
  for i in (seating_arrangement):
    print(i)
    # for j in range(len(seating_arrangement[0])):




In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
# Neural network architecture
class SeatingArrangementNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SeatingArrangementNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.flat = nn.Flatten(0, -1)
    def forward(self, x):
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Hyperparameters
num_people = 400
num_unfriendly_pairs = 600
learning_rate = 0.001
num_epochs = 10
num_samples = 100
group_size = 5
row = 20
col = 20
# Model definition
input_size = num_people * num_people
hidden_size = 24
output_size = num_people # Output represents predicted seating order
model = SeatingArrangementNet(input_size, hidden_size, output_size)
# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = calculate_conflict_cost
# Load training data
num_groups = num_people // group_size
groups = create_groups(num_people, group_size)
# print(groups)
urm_data = generate_training_data(num_people, num_unfriendly_pairs, num_samples, groups)
urm_data = torch.tensor(urm_data).float()
# print(len(urm_data))
# Training loop
for epoch in range(num_epochs):
    for i in range(0, len(urm_data)):
        urm_tensor = torch.tensor(urm_data[i])
        predictions = model(urm_tensor.type(torch.FloatTensor))
        # print("len ",len(predictions))
        # print('predictions', predictions)
        predictions = predictions.argsort(dim=0)
        # permuted_outputs = torch.stack([convert_to_permutation(output) for output in predictions])
        permuted_outputs = convert_to_permutation(predictions)
        # print('permuted', permuted_outputs)
        # int_array = [tensor.item() for tensor in permuted_outputs]
        permuted_outputs = [element - 1 for element in permuted_outputs]
        loss = loss_fn(permuted_outputs, urm_tensor, groups) # Update loss function arguments
        optimizer.zero_grad()
        loss.requires_grad = True
        loss.backward()
        optimizer.step()
        # print("urm[i] is done")

    print(f"Epoch: {epoch+1}/{num_epochs}")
print("Training complete!", )

# test
unfreindly_matrix = generate_unfriendly_matrix(num_people, num_unfriendly_pairs, groups)
test_urm_tensor = torch.tensor(unfreindly_matrix)
predicted_seating_arrangement = model(test_urm_tensor.flatten().float())

predicted_seating_arrangement = convert_to_permutation(predicted_seating_arrangement)
predicted_seating_arrangement = [element - 1 for element in predicted_seating_arrangement]
names = convert_index_to_names(predicted_seating_arrangement, num_people, row, col)
conflict = loss_fn(predicted_seating_arrangement, unfreindly_matrix, groups)
# print("names: ", names, "conflict:", conflict)
_print(names)
print("conflict", conflict)

<ipython-input-5-eb62abf9c8e6>:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  urm_tensor = torch.tensor(urm_data[i])


Epoch: 1/10
Epoch: 2/10
Epoch: 3/10
Epoch: 4/10
Epoch: 5/10
Epoch: 6/10
Epoch: 7/10
Epoch: 8/10
Epoch: 9/10
Epoch: 9/10
Epoch: 10/10
Training complete!
Epoch: 10/10
Training complete!
['Ali Karimi', 'Hossein boushehri', 'Reza Hosseini', 'AmirAbbasHashem Ebrahimi', 'Sobhan Rahimi', 'Khadije boushehri', 'Saeide Rostami', 'AmirHosein Hosseinzadeh', 'AmirAbbasHashem Rahimi', 'Bahare boushehri', 'Kosar Mohammadi', 'Zahra Hosseinzadeh', 'Razie Hosseini', 'AmirMahdi Arjmandi', 'Khadije Mohammadi', 'Reza Fazeli', 'Zahra Rokni', 'Roya Rahimi', 'Ali Rahimi', 'Sobhan Hosseinzadeh']
['Sara Rokni', 'Hossein Niknam', 'Razie Rahimi', 'Hamed Shirazi', 'Kosar Jalali', 'Mohammad Erfani', 'Mehdi Hashemi', 'Bahare Ahmadi', 'Mehdi Karimi', 'Saeide Ebrahimi', 'Hossein Karimi', 'AmirAbbasHashem boushehri', 'Hossein Fazeli', 'Khadije Ebrahimi', 'Hossein Ebrahimi', 'Amir Karimi', 'Ali Hashemi', 'Saeide Jalali', 'Hamid Rahimi', 'Kosar Hosseinzadeh']
['Fatemeh Ahmadi', 'AmirAbbasHashem Arjmandi', 'Fatemeh Rostam